# Яндекс Недвижимость

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

In [2]:
real_estate_data = pd.read_csv('./datasets/real_estate_data.csv', sep='\t')

In [3]:
display(real_estate_data.head())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,NaN,...,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,...,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,...,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,NaN,...,41.0,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


In [4]:
display(real_estate_data['is_apartment'].unique())

array([nan, False, True], dtype=object)

<div class="alert alert-info" role="alert">
    <p class="mb-0">Как получить <code>list</code> колонок с <code>NaN</code> нашёл на <code>stackoverflow</code> 👌</p>
</div>

In [5]:
display(
    real_estate_data.columns[real_estate_data.isna().any()].tolist()
)

['ceiling_height',
 'floors_total',
 'living_area',
 'is_apartment',
 'kitchen_area',
 'balcony',
 'locality_name',
 'airports_nearest',
 'cityCenters_nearest',
 'parks_around3000',
 'parks_nearest',
 'ponds_around3000',
 'ponds_nearest',
 'days_exposition']

Добро, вот с этими колонками нужно разобраться в них пропуски — `NaN`

In [6]:
display(real_estate_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

None

In [7]:

for column in real_estate_data.columns:
    count_of_nan = len(real_estate_data.loc[real_estate_data[column].isna()][column])
    if count_of_nan > 0:
        print(column, end=' NaN = ')
        print(count_of_nan)
        print(real_estate_data[column].describe(), end='\n\n')

ceiling_height NaN = 9195
count    14504.000000
mean         2.771499
std          1.261056
min          1.000000
25%          2.520000
50%          2.650000
75%          2.800000
max        100.000000
Name: ceiling_height, dtype: float64

floors_total NaN = 86
count    23613.000000
mean        10.673824
std          6.597173
min          1.000000
25%          5.000000
50%          9.000000
75%         16.000000
max         60.000000
Name: floors_total, dtype: float64

living_area NaN = 1903
count    21796.000000
mean        34.457852
std         22.030445
min          2.000000
25%         18.600000
50%         30.000000
75%         42.300000
max        409.700000
Name: living_area, dtype: float64

is_apartment NaN = 20924
count      2775
unique        2
top       False
freq       2725
Name: is_apartment, dtype: object

kitchen_area NaN = 2278
count    21421.000000
mean        10.569807
std          5.905438
min          1.300000
25%          7.000000
50%          9.100000
75%         

## Шаг 2. Предобработка данных

Определите и изучите пропущенные значения:
Для некоторых пропущенных значений можно предположить логичную замену. Например, если человек не указал число балконов — скорее всего, их нет. Такие пропуски правильно заменить на 0. Для других типов данных нет подходящего значения на замену. В этом случае правильно оставить эти значения пустыми. Отсутствие значения — тоже важный сигнал, который не нужно прятать.



Заполните пропуски, где это уместно. Опишите, почему вы решили заполнить пропуски именно в этих столбцах и как выбрали значения.
Укажите причины, которые могли привести к пропускам в данных.

In [8]:
real_estate_data['is_apartment'].fillna(value=False, inplace=True)
real_estate_data['balcony'].fillna(value=0, inplace=True)

In [9]:
display(
    real_estate_data.loc[real_estate_data['locality_name']
                         .isna()].head(5)
)
#real_estate_data['locality_name'].fillna(value=False, inplace=True)


,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
1097,3,8600000.0,81.7,2016-04-15T00:00:00,3,3.55,5.0,50.8,2,False,...,8.8,0.0,NaN,23478.0,4258.0,0.0,NaN,0.0,NaN,147.0
2033,6,5398000.0,80.0,2017-05-30T00:00:00,3,NaN,4.0,42.6,2,False,...,18.6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0
2603,20,3351765.0,42.7,2015-09-20T00:00:00,1,NaN,24.0,15.6,3,False,...,10.7,0.0,NaN,22041.0,17369.0,0.0,NaN,1.0,374.0,276.0
2632,2,5130593.0,62.4,2015-10-11T00:00:00,2,NaN,24.0,33.1,21,False,...,8.2,0.0,NaN,22041.0,17369.0,0.0,NaN,1.0,374.0,256.0
3574,10,4200000.0,46.5,2016-05-28T00:00:00,2,NaN,5.0,30.8,5,False,...,6.5,0.0,NaN,27419.0,8127.0,0.0,NaN,1.0,603.0,45.0


In [10]:
display(
    real_estate_data.columns[real_estate_data.isna().any()].tolist()
)

['ceiling_height',
 'floors_total',
 'living_area',
 'kitchen_area',
 'locality_name',
 'airports_nearest',
 'cityCenters_nearest',
 'parks_around3000',
 'parks_nearest',
 'ponds_around3000',
 'ponds_nearest',
 'days_exposition']

    ❌ 'ceiling_height' — сделаем `NaN` медианным значением по району
    ❌ 'floors_total' — сделаем `NaN` медианным значением по району
    ❌ 'living_area' — сделаем `NaN` медианным значением по диапазонам цен и рай
    ✅ 'is_apartment'
    ❌ 'kitchen_area' — если студия то кухня `NaN` проверим, если да то сделаем 0
    ✅ 'balcony'
    ❌ 'locality_name' — сначала удалим неявные дубликаты
    ❌ 'airports_nearest' — проверим по районам, может и нет ближайших аэропортов тогда `NaN` = 0
    ❌ 'cityCenters_nearest' — проверим по районам, может и нет ближайших центров города тогда `NaN` = 0
    ❌ 'parks_around3000' — проверим по районам, может и нет ближайших парков тогда `NaN` = 0
    ❌ 'parks_nearest'
    ❌ 'ponds_around3000' — проверим по районам, может и нет ближайших прудов тогда `NaN` = 0
    ❌ 'ponds_nearest'
    ❌ 'days_exposition' — посмотрим как связан столбец с `first_day_exposition`

**Предполагаю**

что стоит заменить пропущенные высоту потолков и количество этажей на медиану районных значений.

### 'locality_name' — сначала удалим неявные дубликаты

In [ ]:
def correct_name(to_correct):
    pass

In [15]:
display(
    real_estate_data['locality_name'].unique()[:5]
)

array(['Санкт-Петербург', 'посёлок Шушары', 'городской посёлок Янино-1',
       'посёлок Парголово', 'посёлок Мурино'], dtype=object)

In [12]:
display(
    real_estate_data.loc[real_estate_data['locality_name'].isna()]['is_apartment'].unique()
)

array([False])

In [13]:
display(
    real_estate_data.loc[real_estate_data['locality_name'].isna()]['is_apartment'].count()
)

49

Так, похоже, что у всех `'locality_name'` = `NaN`, `'is_apartment'` = `False`

In [14]:
display(
    len(real_estate_data.loc[real_estate_data['parks_around3000'].isna()]['parks_nearest']),
    len(real_estate_data.loc[real_estate_data['parks_around3000'] == 0]['parks_nearest'].unique()),
    (real_estate_data.loc[(real_estate_data['parks_around3000'] == 0)
                             and (real_estate_data['parks_nearest'].isna())]['rooms'].count()),
    
)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
real_estate_data['first_day_exposition'] = pd.to_datetime(real_estate_data['first_day_exposition'], format='%d/%m/%YT%h%m%s')

In [ ]:
display(
    real_estate_data['first_day_exposition'].describe()
)

In [ ]:
display(
    real_estate_data.loc[real_estate_data['locality_name'].isna()].head()
)
#real_estate_data['locality_name'].fillna(value=False, inplace=True)


In [ ]:
display(
    real_estate_data.loc[real_estate_data['cityCenters_nearest'].isna()]['locality_name'].unique(),
    len(real_estate_data.loc[real_estate_data['cityCenters_nearest'].isna()]['locality_name'].unique())
)
#real_estate_data['locality_name'].fillna(value=False, inplace=True)


In [ ]:
display(
    
    real_estate_data['cityCenters_nearest'] = (
        real_estate_data.loc[real_estate_data['cityCenters_nearest'].isna()]['locality_name'].unique()
    ),
    len(real_estate_data.loc[real_estate_data['cityCenters_nearest'].isna()]['locality_name'].unique())
)
#real_estate_data['locality_name'].fillna(value=False, inplace=True)


In [ ]:
display(
    real_estate_data.loc[real_estate_data['studio'] == True].head(),
    len(real_estate_data.loc[real_estate_data['studio'] == True])
)
#real_estate_data['locality_name'].fillna(value=False, inplace=True)


`ceiling_height` — пропуски заменим медианой по району `locality_name`

In [ ]:
display(real_estate_data.info())

Приведите данные к нужным типам:
Поясните, в каких столбцах нужно изменить тип данных и почему.

In [ ]:
real_estate_data.astype(
        {
            'total_images': 'uint16',
            'rooms':'uint8',
            'ceiling_height':'float16',
            'floors_total':'uint16',
            'floor':'int16',
            'parks_around3000':'uint8',
            'ponds_around3000':'uint8',
            'is_apartment': 'bool',
        }
    ).dtypes


## Шаг 3. Посчитайте и добавьте в таблицу

цену квадратного метра;
день недели, месяц и год публикации объявления;
этаж квартиры; варианты — первый, последний, другой;
соотношение жилой и общей площади, а также отношение площади кухни к общей.

## Шаг 4. Проведите исследовательский анализ данных

Изучите следующие параметры: площадь, цена, число комнат, высота потолков. Постройте гистограммы для каждого параметра.
Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?
Уберите редкие и выбивающиеся значения. Опишите, какие особенности обнаружили.
Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от площади, числа комнат, удалённости от центра. Изучите зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом. Также изучите зависимость от даты размещения: дня недели, месяца и года.
Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите среди них населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце locality_name.
Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге (locality_name). Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется, — это и будет центральная зона.
Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всей базе?

## Шаг 5. Напишите общий вывод

## Описание данных

`airports_nearest` — расстояние до ближайшего аэропорта в метрах (м)

`balcony` — число балконов

`ceiling_height` — высота потолков (м)

`cityCenters_nearest` — расстояние до центра города (м)

`days_exposition` — сколько дней было размещено объявление (от публикации до снятия)

`first_day_exposition` — дата публикации

`floor` — этаж

`floors_total` — всего этажей в доме

`is_apartment` — апартаменты (булев тип)

`kitchen_area` — площадь кухни в квадратных метрах (м²)

`last_price` — цена на момент снятия с публикации

`living_area` — жилая площадь в квадратных метрах (м²)

`locality_name` — название населённого пункта

`open_plan` — свободная планировка (булев тип)

`parks_around3000` — число парков в радиусе 3 км

`parks_nearest` — расстояние до ближайшего парка (м)

`ponds_around3000` — число водоёмов в радиусе 3 км

`ponds_nearest` — расстояние до ближайшего водоёма (м)

`rooms` — число комнат

`studio` — квартира-студия (булев тип)

`total_area` — площадь квартиры в квадратных метрах (м²)

`total_images` — число фотографий квартиры в объявлении